In [ ]:
# %%
from paraview.simple import *

def create_field_visualization(field_name, pvd_path, color_range, output_path):
    """
    Create and save a visualization for a given field.
    
    Args:
        field_name (str): Name of the field to visualize
        pvd_path (str): Path to the PVD file
        color_range (tuple): Min and max values for the color scale
        output_path (str): Path to save the output image
    """
    # Load the PVD file
    data = OpenDataFile(pvd_path)
    data.UpdatePipeline()
    
    # Step 1: Open the PVD
    data = OpenDataFile(pvd_path)
    data.UpdatePipeline()

    # Step 2: Determine how to compute magnitude
    info = data.GetPointDataInformation()
    array_info = info.GetArray(field_name)
    num_components = array_info.GetNumberOfComponents()
    calc = Calculator(Input=data)
    calc.ResultArrayName = "Abs"
    calc.Function = f"abs({field_name})" if num_components == 1 else f"sqrt({field_name}_0^2 + {field_name}_1^2)"
    calc.UpdatePipeline()

    # Step 4: Convert to cell data
    p2c = PointDatatoCellData(Input=calc)
    p2c.PassPointData = 1
    p2c.UpdatePipeline()

    # Step 5: Integrate the result
    integrator = IntegrateVariables(Input=p2c)
    integrator.UpdatePipeline()

    # Step 6: Fetch result
    from paraview.vtk.util.numpy_support import vtk_to_numpy
    from paraview.simple import servermanager

    output = servermanager.Fetch(integrator)
    cell_data = output.GetCellData()
    print("======================")
    for i in range(cell_data.GetNumberOfArrays()):
        name = cell_data.GetArrayName(i)
        arr = vtk_to_numpy(cell_data.GetArray(i))
        if name == "Abs":
            print(f"{field_name} - {name}: {arr}")
    
    # Create a render view and show data
    renderView = CreateRenderView()
    renderView.OrientationAxesVisibility = 0
    renderView.Background = [1.0, 1.0, 1.0]
    GetRenderView().UseColorPaletteForBackground = 0
    dataDisplay = Show(data, renderView)
    ResetCamera()
    
    # Zoom in the view by adjusting the camera position
    camera = GetActiveCamera()
    camera.Zoom(3.7)  # Zoom in by a factor of 2.0

    # Get current position and focal point
    position = list(camera.GetPosition())
    focal_point = list(camera.GetFocalPoint())

    # Apply shift to X components
    position[0] += (-1.75)
    focal_point[0] += (-1.75)
    # Apply shift to X components
    position[1] += (0.25)
    focal_point[1] += (0.25)

    # Set the new camera position and focal point
    camera.SetPosition(position)
    camera.SetFocalPoint(focal_point)
    

    # Color by the field
    ColorBy(dataDisplay, ('POINTS', field_name))

    # Rescale color transfer function
    lut = GetColorTransferFunction(field_name)
    lut.RescaleTransferFunction(*color_range)

    # Render and save
    Render()
    SaveScreenshot(output_path, renderView, ImageResolution=[1000, 1400])

def create_colorbar(field_name, color_range, title, output_path):
    """
    Create and save a colorbar for a given field.
    
    Args:
        field_name (str): Name of the field
        color_range (tuple): Min and max values for the color scale
        title (str): Title for the colorbar
        output_path (str): Path to save the output image
    """
    # Create a render view
    renderView = CreateRenderView()
    renderView.OrientationAxesVisibility = 0
    renderView.Background = [1.0, 1.0, 1.0]
    GetRenderView().UseColorPaletteForBackground = 0

    # Create the color transfer function
    lut = GetColorTransferFunction(field_name)
    lut.RescaleTransferFunction(*color_range)

    # Create and configure the scalar bar
    scalarBar = CreateScalarBar(LookupTable=lut, Title=title, ComponentTitle='',
                               TitleFontSize=10, LabelFontSize=10, 
                               TitleColor=[0, 0, 0], LabelColor=[0, 0, 0])
    scalarBar.ScalarBarLength = 0.96
    scalarBar.Position = [0.1, 0.1]

    # Add the scalar bar to the view
    renderView.Representations.append(scalarBar)

    # Render and save
    Render()
    SaveScreenshot(output_path, renderView, ImageResolution=[380, 1400])


In [ ]:

# Create all visualizations
# B_tor fields
create_field_visualization('B_tor_CG', 'ParaView/B_tor_CG/B_tor_CG.pvd', (-12.0, -3.3), 'png_output/B_tor_CG.png')
create_field_visualization('B_tor_DG', 'ParaView/B_tor_DG/B_tor_DG.pvd', (-12.0, -3.3), 'png_output/B_tor_DG.png')
create_colorbar('B_tor_CG', (-12.0, -3.3), 'Toroidal Component of the Magnetic Field', 'png_output/B_tor_colorbar.png')

# B_pol fields
create_field_visualization('B_pol_Hdiv', 'ParaView/B_pol_Hdiv/B_pol_Hdiv.pvd', (0, 2.5), 'png_output/B_pol_Hdiv.png')
create_field_visualization('B_pol_Hcurl', 'ParaView/B_pol_Hcurl/B_pol_Hcurl.pvd', (0, 2.5), 'png_output/B_pol_Hcurl.png')
create_field_visualization('B_pol_vec_CG', 'ParaView/B_pol_vec_CG/B_pol_vec_CG.pvd', (0, 2.5), 'png_output/B_pol_vec_CG.png')
create_colorbar('B_pol_Hdiv', (0, 2.5), 'Poloidal Component of the Magnetic Field', 'png_output/B_pol_colorbar.png')

# J_tor fields
create_field_visualization('J_tor_Hdiv', 'ParaView/J_tor_Hdiv/J_tor_Hdiv.pvd', (-0.3, 1.4), 'png_output/J_tor_Hdiv.png')
create_field_visualization('J_tor_Hcurl', 'ParaView/J_tor_Hcurl/J_tor_Hcurl.pvd', (-0.3, 1.4), 'png_output/J_tor_Hcurl.png')
create_field_visualization('J_tor_direct', 'ParaView/J_tor_direct/J_tor_direct.pvd', (-0.3, 1.4), 'png_output/J_tor_direct.png')
create_field_visualization('J_tor_vec_CG', 'ParaView/J_tor_vec_CG/J_tor_vec_CG.pvd', (-0.3, 1.4), 'png_output/J_tor_vec_CG.png')
create_colorbar('J_tor_Hdiv', (-0.3, 1.4), 'Toroidal Component of the Current Density', 'png_output/J_tor_colorbar.png')

# J_pol fields
create_field_visualization('J_pol_Hdiv', 'ParaView/J_pol_Hdiv/J_pol_Hdiv.pvd', (0, 0.24), 'png_output/J_pol_Hdiv.png')
create_field_visualization('J_pol_Hcurl', 'ParaView/J_pol_Hcurl/J_pol_Hcurl.pvd', (0, 0.24), 'png_output/J_pol_Hcurl.png')
create_field_visualization('J_pol_direct', 'ParaView/J_pol_direct/J_pol_direct.pvd', (0, 0.24), 'png_output/J_pol_direct.png')
create_field_visualization('J_pol_vec_CG', 'ParaView/J_pol_vec_CG/J_pol_vec_CG.pvd', (0, 0.24), 'png_output/J_pol_vec_CG.png')
create_colorbar('J_pol_Hdiv', (0, 0.24), 'Poloidal Component of the Current Density', 'png_output/J_pol_colorbar.png')

# JxB_pol fields
create_field_visualization('JxB_pol_A', 'ParaView/JxB_pol_A/JxB_pol_A.pvd', (0, 0.1), 'png_output/JxB_pol_A.png')
create_field_visualization('JxB_pol_B', 'ParaView/JxB_pol_B/JxB_pol_B.pvd', (0, 0.1), 'png_output/JxB_pol_B.png')
create_field_visualization('JxB_pol_C', 'ParaView/JxB_pol_C/JxB_pol_C.pvd', (0, 0.1), 'png_output/JxB_pol_C.png')
create_colorbar('JxB_pol_A', (0, 0.1), 'Poloidal Component of the Lorentz Force', 'png_output/JxB_pol_colorbar.png')

# JxB_tor fields
create_field_visualization('JxB_tor_A', 'ParaView/JxB_tor_A/JxB_tor_A.pvd', (-0.01, 0.01), 'png_output/JxB_tor_A.png')
create_field_visualization('JxB_tor_B', 'ParaView/JxB_tor_B/JxB_tor_B.pvd', (-0.01, 0.01), 'png_output/JxB_tor_B.png')
create_field_visualization('JxB_tor_C', 'ParaView/JxB_tor_C/JxB_tor_C.pvd', (-0.01, 0.01), 'png_output/JxB_tor_C.png')
create_colorbar('JxB_tor_A', (-0.01, 0.01), 'Toroidal Component of the Lorentz Force', 'png_output/JxB_tor_colorbar.png')

# div_B_p fields
create_field_visualization('div_B_pol_Hdiv', 'ParaView/div_B_pol_Hdiv/div_B_pol_Hdiv.pvd', (-0.1, 0.1), 'png_output/div_B_pol_Hdiv.png')
create_field_visualization('div_B_pol_Hcurl', 'ParaView/div_B_pol_Hcurl/div_B_pol_Hcurl.pvd', (-0.1, 0.1), 'png_output/div_B_pol_Hcurl.png')
create_field_visualization('div_B_pol_vec_CG', 'ParaView/div_B_pol_vec_CG/div_B_pol_vec_CG.pvd', (-0.1, 0.1), 'png_output/div_B_pol_vec_CG.png')
create_colorbar('div_B_pol_Hdiv', (-0.1, 0.1), 'Divergence of the Poloidal Component of the Magnetic Field', 'png_output/div_B_pol_colorbar.png')
